In [1]:
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import pickle
from os import path
import sys
sys.path.append("../..")
import util
import tqdm
import numpy as np
import time

In [2]:
DATA_DIR = "../../../../data/bughouse-db"

In [3]:
filename = "export2018"
file = path.join(DATA_DIR, "bpgn", f"{filename}.bpgn")
filepath = path.join(DATA_DIR, "pk", f"{filename}.pk")

In [4]:
def print_insights(reg, X, y, label=""):
    preds = reg.predict(X)
    print(f"{label} predictions: min = {np.min(preds)}, max = {np.max(preds)}, mean = {np.mean(preds)}, mean diff from mean: {np.mean(preds - np.mean(preds))}")
    score = reg.score(X, y)
    rmse = np.sqrt(np.mean((preds - y) ** 2))
    print(f"{label}: score = {score}, RMSE = {rmse}")

In [5]:
def save_model(model, save_path):
    with open(save_path, 'wb') as f:
        pickle.dump(model, f)

In [6]:
print("Loading dataset")
with open(filepath, 'rb') as f:
    data_dict = pickle.load(f)

Loading dataset


In [ ]:
print("Processing dataset")
states, values = util.dataset_to_array(data_dict, tqdm.tqdm, max_n=500000)
print(f"Dataset memory: {sys.getsizeof(states) / 1048576} mb")

Processing dataset


 13%|████████▉                                                            | 64801/500000 [00:32<03:35, 2018.52it/s]

In [ ]:
split_n = int(0.85 * states.shape[0])
indices = np.random.permutation(states.shape[0])
training_idx, test_idx = indices[:split_n], indices[split_n:]
X_train, X_test = states[training_idx, :], states[test_idx, :]
y_train, y_test = values[training_idx], values[test_idx]

In [ ]:
t = time.time()
print("Fitting linear regression")
reg = LinearRegression().fit(X_train, y_train)
print(f"Time taken: {time.time() - t}")

In [ ]:
print_insights(reg, X_train, y_train, "Train")
print_insights(reg, X_test, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/LinearRegression.sav')

with open('saved_models/LinearRegression.sav', 'rb') as f:
   loaded_model = pickle.load(f)
diff = loaded_model.predict(X_test) - reg.predict(X_test)
assert np.abs(np.sum(diff)) < 1e-3

In [ ]:
print("Fitting ElasticNet")
reg = ElasticNet().fit(X_train, y_train)

In [ ]:
print_insights(reg, X_train, y_train, "Train")
print_insights(reg, X_test, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/ElasticNet.sav')

In [ ]:
print("Fitting DecisionTreeRegressor")
reg = DecisionTreeRegressor().fit(X_train, y_train)

In [ ]:
print_insights(reg, X_train, y_train, "Train")
print_insights(reg, X_test, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/DecisionTreeRegessor.sav')

In [ ]:
print("Fitting RandomForestRegressor")
reg = RandomForestRegressor(n_estimators=10).fit(X_train, y_train)

In [ ]:
print_insights(reg, X_train, y_train, "Train")
print_insights(reg, X_test, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/RandomForestRegressor.sav')

In [ ]:
# print("Fitting SVR")
# reg = SVR().fit(X_train, y_train)

In [ ]:
# print_insights(reg, X_train, y_train, "Train")
# print_insights(reg, X_test, y_test, "Test")

In [ ]:
# save_model(reg, 'saved_models/SVR.sav')

# Models with PCA

In [ ]:
svd = TruncatedSVD(n_components=512)
X_train_PCA = svd.fit_transform(X_train)
print("Fit PCA")
X_test_PCA = svd.transform(X_test)
print(f"PCA explaination: {np.sum(svd.explained_variance_ratio_)}")

In [ ]:
with open('saved_models/PCA.sav', 'rb') as f:
    svd = pickle.load(f)
X_train_PCA = svd.transform(X_train)
X_test_PCA = svd.transform(X_test)

In [ ]:
save_model(svd, 'saved_models/PCA.sav')

In [ ]:
print("Fitting linear regression")
reg = LinearRegression().fit(X_train_PCA, y_train)

In [ ]:
print_insights(reg, X_train_PCA, y_train, "Train")
print_insights(reg, X_test_PCA, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/LinearRegressionPCA.sav')

In [ ]:
print("Fitting ElasticNet")
reg = ElasticNet().fit(X_train_PCA, y_train)

In [ ]:
print_insights(reg, X_train_PCA, y_train, "Train")
print_insights(reg, X_test_PCA, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/ElasticNetPCA.sav')

In [ ]:
print("Fitting DecisionTreeRegression")
reg = DecisionTreeRegressor().fit(X_train_PCA, y_train)

In [ ]:
print_insights(reg, X_train_PCA, y_train, "Train")
print_insights(reg, X_test_PCA, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/DecisionTreeRegessorPCA.sav')

In [ ]:
print("Fitting RandomForestRegressorPCA")
reg = RandomForestRegressor(n_estimators=10).fit(X_train_PCA, y_train)

In [ ]:
print_insights(reg, X_train_PCA, y_train, "Train")
print_insights(reg, X_test_PCA, y_test, "Test")

In [ ]:
save_model(reg, 'saved_models/RandomForestRegressorPCA.sav')

In [ ]:
# print("Fitting SVR")
# reg = SVR().fit(X_train_PCA, y_train)

In [ ]:
# print_insights(reg, X_train_PCA, y_train, "Train")
# print_insights(reg, X_test_PCA, y_test, "Test")

In [ ]:
# save_model(reg, 'saved_models/SVR.sav')